# gpt-oss

- [モデルカード][1]
- [GitHub][2]
- [ウェブサイト][3]

[1]: https://arxiv.org/abs/2508.10925
[2]: https://github.com/openai/gpt-oss
[3]: https://openai.com/index/introducing-gpt-oss/

## 導入

gpt-ossは、120bと20bの2つのオープンウェイト推論モデル

Response APIと互換性があり、エージェントワークフローに組み込める:

- 高い指示追従性
- ウェブ検索やコード実行などのツール仕様
- 推論の深さを設定できる機能
- CoT（chain-of-thought, 思考の連鎖）対応
- JSON形式などの出力に対応

Preparedness Frameworkで危険性を評価:

- Preparedness Frameworkは、モデルの高度の能力を追跡しリスクを見積もる工程
- 以下の分野で高リスクでないことを確認
    - 生物・化学（Biological and Chemical capability）
        - モデルが生物兵器・化学兵器を助長する能力を持つかどうか
    - サイバー（Cyber capability）
        - システムの機密性・完全性・可用性を破壊する能力を持つかどうか
    - AI自己改善（AI Self-Improvement）
        - モデルが再帰的な能力改善を引き起こす能力を持つかどうか

## アーキテクチャ

特徴:

- GPT-2・GPT-3を基に構築
- 自己回帰型のMixture-of-Experts（MoE）Transformerモデル
- gpt-oss-120b: 36層
- gpt-oss-20b: 24層

![](image/arch.png)

事後学習で、MoEのパラメータをMXFP4に量子化してファインチューニング:

- MoEのパラメータは総パラメータ数の90%以上を占めている
- 量子化でパラメータあたり4.25ビットに削減
    - 120bモデルは単一の80GBのVRAMで推論可能
    - 20bモデルは16GBのVRAMで推論可能

アーキテクチャ詳細:

- 2880次元の残差接続（residual stream）を持つ
- アテンションおよびMoEを適用する前にRMSNormを適用（Pre-LN）

MoEブロック:

- 一般的な線形ルータプロジェクション（linear router projection）を採用
    - 120bモデルは128個のエキスパート
    - 20bモデルは32個のエキスパート
- トークンごとに4つのエキスパートを選択し、Softmaxで重み付け
- MoEブロックは、Gated SwiGLU活性化関数を採用
    - 値をクリップしたり（clamping）、残差接続を加えるなどカスタマイズしたSwiGLUを使用

アテンション機構:

- 2種類のアテンション機構を交互に繰り返す
    - ウィンドウサイズが128トークンのアテンション（banded window pattern）
    - フルアテンション（fully dense pattern）
- GQA（Grouped Query Attention）でメモリ使用量と推論時間を削減
    - ヘッドは64次元
    - クエリヘッドは64個 
    - キー・バリューヘッドは8個
- RoPE（Rotary Position Embeddings）を使用
- YaRN（Yet another Rope extentioN）でフルアテンションのコンテキスト長を131,072トークンに拡張

各アテンションヘッドのSoftmaxの分母に学習可能なバイアス項を追加:

- [off-by-one attention][1]や[attention sinks][2]から着想
    - 従来のアテンションヘッドは「どのトークンにも注意を払わない」選択ができなかった
    - バイアス項を加えることで注意を払わない選択が可能になり、学習が安定化する

[1]: https://www.evanmiller.org/attention-is-off-by-one.html
[2]: https://arxiv.org/abs/2309.17453

AIME・GPQA Diamond・HLE・MMLUベンチマークでo4-miniに匹敵する性能:

![](image/fig1.png)

## トークナイザー

TikTokenライブラリで交際されている[o200k_harmony][1]トークナイザーを使用:

- BPE（Byte Pair Encoding）
- GPT-4oやo4-miniと同様
- Harmonyチャットフォーマットに互換
- 語彙数は201,088トークン

![](image/fig17.png)

[1]: https://github.com/openai/tiktoken

## 事前学習

訓練に数兆トークンのデータセットを使用:

- STEM（科学・技術・工学・数学）、コーディング、一般常識
- GPT-4oのCBRN事前学習フィルタを再利用し安全性を向上
    - CBRN: Chemical・Biological・Radiodical・Nuclear
- 知識カットオフは2024年6月

NVIDIA H100 GPUで訓練:

- PyTorchと、エキスパートの最適化にTritonを使用
- Flash Attentionでメモリ使用量を削減し、訓練を高速化
- gpt-oss-120bの訓練は、H100で210万時間に相当する計算量
- gpt-oss-20bの訓練は、H100で21万時間に相当する計算量

## 事後学習

事後学習は、OpenAI O3と同様のCoT RL手法で実施:

- CoTによる推論の強化
- ツールの使い方を習得
- ChatGPTと似た個性
- データセットは、コーディング・数学・科学などの幅広い分野

Harmonyチャットフォーマットを使用し、OpenAI APIと互換:

- SystemロールやDeveloperロールなどのロールベースのチャットフォーマット
- 指示が競合した場合、System > Developer > User > Assistant > Toolという優先順位で処理
- CoT用のanalysisチャネル、関数ツール呼び出し用のcommentaryチャネル、最終解答用のfinalチャネルなども含む
- 詳細は[openai/harmony](https://github.com/openai/harmony)

可変な推論レベルの訓練:

- `Reasoning: low` のようなキーワードをシステムプロンプトに追加すると、CoTの長さを変更できる

モデルに自律的にツールを使用するように訓練:

- ブラウジングツール
    - `search`および`open`関数を呼び出し、解答の事実性や知識のカットオフに対処
- Pythonツール
    - ステートフルなJupyter Notebook環境でコードを実行
- 任意のスキーマのツール
    - OpenAI APIと同様に任意の関数の実行
    - 詳細は[openai/gpt-oss](https://github.com/openai/gpt-oss)

![](image/fig18.png)

## 評価

推論レベルを「高」にしてpass@1で評価:

- 推論と事実性
    - AIME（数学コンテスト）
    - GPQA（博士レベルの科学）
    - MMLU（大学レベル試験）
    - HLE（専門家レベル試験）
- コーディング
    - Codex CLIと似たツールへのアクセスの有無を分けて評価
    - Codeforces Elo（競技プログラミング）
    - SWE-bench Verified（ソフトウェア工学）
- ツール使用
    - $\tau$-Bench Retail（小売業タスク）
- その他
    - MMMLU（多言語の大学レベル試験）
    - HealthBench（健康分野）

コーディングとツール使用におけるモデル性能はo4-miniに匹敵する性能:

![](image/fig2.png)

推論レベルを高くすると、CoTが長くなり回答の精度が上がる:

![](image/fig3.png)

推論レベルが高いgpt-oss-120bは、OpenAI o3に匹敵する性能:

![](image/fig4.png)

多言語性能は推論レベルの高いgpt-oss-120bで、o3-mini程度の性能:

![](image/table2.png)

全体的には推論レベルが上がると性能が向上:

![](image/table3.png)

## 安全性テストとリスク緩和

「悪意のある開発者がモデルをファインチューニングする可能性」を考慮してテスト

Preparedness Frameworkで、生物・科学、サイバー、自己改善の分野で高いリスクではないことを確認

更に2つの観点で調査:

1. 悪意のある開発者がgpt-oss-120bを追加学習して、生物・科学、サイバー領域で高い能力を実現できるか
    - SAG（Safety Advisory Group）がファインチューニングを行っても問題なかった
2. gpt-oss-120bをリリースすることで、生物分野の研究を著しく前進できるか
    - 同等性能のオープンウェイトモデルがすでに存在するので実現可能性は低い

## ファインチューニングしていないモデルの安全性

標準的なベンチマークによる不許可コンテンツに対する安全性は飽和:

![](image/table4.png)

マルチターンのベンチマークでは、o4-mini以上の性能:

![](image/table5.png)

[StrongReject][1]によるジェイルブレイクに対するベンチマークでは、o4-miniと同等の性能:

![](image/table6.png)

[1]: https://proceedings.neurips.cc/paper_files/paper/2024/hash/e2e06adf560b0706d3b1ddfca9f29756-Abstract-Datasets_and_Benchmarks_Track.html

システムプロンプトの抽出、プロンプトインジェクションに対する堅牢性はo4-miniより弱い:

![](image/table7.png)

開発者ロールとユーザーロール間のフレーズ保護に対する堅牢性は低い:

![](image/table8.png)

ハルシネーションは、内部に世界知識が少ないgpt-oss-20bは起こしやすい:

![](image/table9.png)

偏見に対する評価はo4-miniと同等:

![](image/table10.png)

## Preparedness Framework

Preparedness Frameworkは、モデルを公開することによる深刻なリスクを追跡し最小限にするフレームワーク

生物・科学領域とサイバー領域では高リスクの可能性があるため、敵対的ファインチューニング（AFT）を実施:

- Helpful-only訓練
    - モデルのガードレールを意図的に解除するための強化学習
- 生物・サイバー領域における能力の最大化
    - 生物領域は、人間の専門家を活用して訓練
        - 質問回答では高い性能に達したが、実験のトラブルシューティング・実験の暗黙知に関しては達しなかった
        - Qwen3やKimi K2はそれと同等の能力を持っているため、リリースしてもリスクは少ない
    - サイバー領域は、CTF（セキュリティ競技）データを活用して訓練

生物領域での危険性の5段階で評価したところ、Helpful-only訓練したモデルは他のモデルと同等の性能:

1. Ideation（アイデア出し）
2. Acquisition（入手）
3. Magnification（増強）
4. Formulation（製剤化）
5. Release（放出）

![](image/fig5.png)

マルチモーダルなウィルス学分野での性能は、o3に劣る性能（ドメインブロック・ブラウジングあり）:

![](image/fig6.png)

実験のトラブルシューティングに関するベンチマークでは、o3に劣る性能:

![](image/fig7.png)

実験の暗黙知とトラブルシューティングに関する別のベンチマークでは、o3に劣る性能:

![](image/fig8.png)

大学生レベルのCTF（Capture the Flag, サイバーセキュリティ）では、o3に劣る性能:

![](image/fig10.png)

プロレベルのCTFベンチマークでは、CTF問題でファインチューニングしたモデルがo3に匹敵する性能:

![](image/fig11.png)

サイバー攻撃に関するベンチマークでは、いずれのシナリオの解決にも至らなかった:

![](image/fig12.png)

AIの自己改善リスクで、GitHubのバグ修正はo4-miniに劣る性能:

![](image/fig19.png)

最先端のAIの研究の再現する能力は、o4-miniに劣る性能:

![](image/fig16.png)

## 実装

In [ ]:
%pip install -qU transformers==4.57.1 kernels

import os
import logging as logging_
import transformers
from transformers import PretrainedConfig
from transformers.utils import logging

from typing import Callable, Optional, Union

import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# トークナイザー

import os
from shutil import copyfile
from typing import TYPE_CHECKING, Any, Optional

# モデル

from typing import Callable, Optional, Union

import torch
from torch import nn

assert torch.cuda.is_available(), "CUDAを使用できません"

from typing import Callable, Optional, Union

import torch
from torch import nn
from torch.nn import functional as F

from transformers.cache_utils import Cache, DynamicCache
from transformers.generation import GenerationMixin
from transformers.integrations.hub_kernels import use_kernel_forward_from_hub
from transformers.masking_utils import create_causal_mask, create_sliding_window_causal_mask
from transformers.modeling_layers import (
    GenericForSequenceClassification,
    GenericForTokenClassification,
    GradientCheckpointingLayer,
)
from transformers.modeling_outputs import MoeCausalLMOutputWithPast, MoeModelOutputWithPast
from transformers.modeling_rope_utils import ROPE_INIT_FUNCTIONS, dynamic_rope_update
from transformers.modeling_utils import ALL_ATTENTION_FUNCTIONS, PreTrainedModel
from transformers.processing_utils import Unpack
from transformers.utils import TransformersKwargs, auto_docstring, can_return_tuple
from transformers.utils.deprecation import deprecate_kwarg
from transformers.utils.generic import OutputRecorder, check_model_inputs
from transformers.models.gpt_oss.configuration_gpt_oss import GptOssConfig

# デバイス設定

torch.set_default_device("cuda")

# ログ設定

if os.path.exists('debug.log'):
    os.remove('debug.log')

def custom_format(record):
    match record.levelno:
        case logging_.DEBUG:
            level = '🟦'
        case logging_.INFO:
            level = '🟩'
        case logging_.WARNING:
            level = '🟨'
        case logging_.ERROR:
            level = '🟥'
        case logging_.CRITICAL:
            level = '🛑'
    return f"{level} {record.getMessage()}"

logging.set_verbosity_debug()
logger = logging.get_logger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging_.Formatter()
formatter.format = custom_format

file_handler = logging_.FileHandler('debug.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging_.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.info(f"Transformers version: {transformers.__version__}")
logger.info(f"Numpy version: {np.__version__}")

### GptOssRMSNorm

RMSNorm（Root Mean Square Layer Normalization）を実装したクラス

In [ ]:
# RMSNormのカーネルを使用（https://huggingface.co/docs/kernels/main/en/index）
@use_kernel_forward_from_hub("RMSNorm")
class GptOssRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        GptOssRMSNorm is equivalent to T5LayerNor）
        """
        logger.info(f"RMSNormを初期化開始 {hidden_size=}, {eps=}")
        super().__init__()
        
        # 1.0に初期化
        # 2880
        self.weight = nn.Parameter(torch.ones(hidden_size))

        # 1e-05
        self.variance_epsilon = eps

        logger.info("RMSNormの初期化完了")

    def forward(self, hidden_states):
        logger.info(f"RMSNormの順伝播開始 {hidden_states.shape=}, {hidden_states.dtype=}")

        # 1. 単精度にアップキャスト
        # bfloat16 -> float32
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        logger.debug(f"アップキャスト {input_dtype=} {hidden_states.dtype=}")

        # 2. 二乗平均を計算（分散）
        # (1, 69, 2880) -> (1, 69, 1)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)

        # 3. 平方根の逆数で正規化
        # (1, 69, 2880) * (1, 69, 1) -> (1, 69, 2880)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)

        # 4. 学習可能な重み（ゲイン）を適用し、元のデータ型に戻す
        # (1, 69, 2880) -> (1, 69, 2880)
        # float32 -> bfloat16
        res = (self.weight * hidden_states).to(input_dtype)  # main diff with Llama

        logger.info(f"RMSNormの順伝播完了 {res.shape=}, {res.dtype=}")
        return res

    def extra_repr(self):
        return f"{tuple(self.weight.shape)}, eps={self.variance_epsilon}"

### GptOssExperts

GptOssExpertsは、MoEのエキスパート層を実装するクラス

In [ ]:
class GptOssExperts(nn.Module):
    def __init__(self, config):
        logger.info(f"GptOssExpertsを初期化開始 {config.intermediate_size=}, {config.num_local_experts=}, {config.hidden_size=}")
        super().__init__()

        # 2880
        self.intermediate_size = config.intermediate_size

        # 32
        self.num_experts = config.num_local_experts

        # 2880
        self.hidden_size = config.hidden_size

        # 2880
        self.expert_dim = self.intermediate_size

        # ゲートプロジェクションとアッププロジェクションのパラメータを初期化
        # (32, 2880, 5760)
        self.gate_up_proj = nn.Parameter(torch.empty(self.num_experts, self.hidden_size, 2 * self.expert_dim))

        # ゲートプロジェクションとアッププロジェクションのバイアスを初期化
        # (32, 5760)
        self.gate_up_proj_bias = nn.Parameter(torch.empty(self.num_experts, 2 * self.expert_dim))

        # ダウンプロジェクションのパラメータを初期化
        # (32, 5760, 2880)
        self.down_proj = nn.Parameter(torch.empty((self.num_experts, self.expert_dim, self.hidden_size)))

        # ダウンプロジェクションのバイアスを初期化
        # (32, 2880)
        self.down_proj_bias = nn.Parameter(torch.empty(self.num_experts, self.hidden_size))

        # カスタムSwiGLUのパラメータ
        self.alpha = 1.702
        self.limit = 7.0
        logger.info("GptOssExpertsの初期化完了")

    def forward(self, hidden_states: torch.Tensor, router_indices=None, routing_weights=None) -> torch.Tensor:
        """
        When training it is more efficient to just loop over the experts and compute the output for each expert
        as otherwise the memory would explode.

        For inference we can sacrifice some memory and compute the output for all experts at once. By repeating the inputs.

        Args:
            hidden_states (torch.Tensor): (batch_size, seq_len, hidden_size)
            selected_experts (torch.Tensor): (batch_size * token_num, top_k)
            routing_weights (torch.Tensor): (batch_size * token_num, num_experts)
        Returns:
            torch.Tensor
        """
        logger.info(f"GptOssExpertsの順伝播開始 {hidden_states.shape=}, {hidden_states.dtype=}, {router_indices.shape=}, {routing_weights.shape=}")

        batch_size = hidden_states.shape[0]
        hidden_states = hidden_states.reshape(-1, self.hidden_size)  # (num_tokens, hidden_size)
        logger.debug(f"{hidden_states.shape=}")

        num_experts = routing_weights.shape[1]

        # CPUまたは訓練時の場合
        if hidden_states.device.type == "cpu" or self.training:

            # 結果を格納するアキュームレータを初期化
            next_states = torch.zeros_like(hidden_states, dtype=hidden_states.dtype, device=hidden_states.device)
            logger.debug(f"{next_states.shape=}")

            with torch.no_grad():
                # エキスパートの割り当てをワンホットエンコーディング
                expert_mask = torch.nn.functional.one_hot(
                    router_indices, num_classes=num_experts + 1
                )  # masking is also a class
                logger.debug(f"{expert_mask.shape=}")

                expert_mask = expert_mask.permute(2, 1, 0)
                logger.debug(f"{expert_mask.shape=}")

                # 実際に1回でも選ばれたエキスパートを取得
                # we sum on the top_k and on the sequence length to get which experts
                # are hit this time around
                expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
                logger.debug(f"{expert_hit.shape=}")

            # 選ばれたエキスパートごとにループ
            for expert_idx in expert_hit[:]:
                # expert_idx only have 1 element, so we can use scale for fast indexing
                expert_idx = expert_idx[0]
                logger.debug(f"エキスパート処理中 {expert_idx=}")

                # skip masking index
                if expert_idx == num_experts:
                    continue

                with torch.no_grad():
                    # 処理するトークンのインデックスを抽出
                    _, token_idx = torch.where(expert_mask[expert_idx])
                    logger.debug(f"{token_idx.shape=}")

                # エキスパートが処理するトークンを抽出
                current_state = hidden_states[token_idx]
                logger.debug(f"{current_state.shape=}")

                # ゲートプロジェクションとアッププロジェクションを同時に適用
                gate_up = current_state @ self.gate_up_proj[expert_idx] + self.gate_up_proj_bias[expert_idx]
                logger.debug(f"{gate_up.shape=}")
                gate, up = gate_up[..., ::2], gate_up[..., 1::2]
                logger.debug(f"{gate.shape=}, {up.shape=}")

                # ゲートプロジェクションの値をクリップ
                gate = gate.clamp(min=None, max=self.limit)

                # アッププロジェクションの値をクリップ
                up = up.clamp(min=-self.limit, max=self.limit)

                # alphaでスケーリングしたカスタムSwiGLUのゲート計算を実行
                glu = gate * torch.sigmoid(gate * self.alpha)

                # アッププロジェクションに1を足してから、ゲートを掛け合わせる（残差接続と等価）
                gated_output = (up + 1) * glu

                # ダウンプロジェクションを適用
                out = gated_output @ self.down_proj[expert_idx] + self.down_proj_bias[expert_idx]
                logger.debug(f"{out.shape=}")

                # ルーティング重みで出力を重み付け
                weighted_output = out * routing_weights[token_idx, expert_idx, None]
                logger.debug(f"{weighted_output.shape=}")

                # アキュームレータに加算
                next_states.index_add_(0, token_idx, weighted_output.to(hidden_states.dtype))

            next_states = next_states.view(batch_size, -1, self.hidden_size)

        # GPUかつ推論時の場合
        else:
            # 入力をエキスパート数分繰り返す
            hidden_states = hidden_states.repeat(num_experts, 1)
            logger.debug(f"{hidden_states.shape=}")

            # エキスパートごとにバッチを分割
            hidden_states = hidden_states.view(num_experts, -1, self.hidden_size)
            logger.debug(f"{hidden_states.shape=}")

            # バッチ行列積（bmm）を使用して全てのエキスパートを並列処理
            # かつ、ゲートプロジェクションとアッププロジェクションを同時に適用
            gate_up = torch.bmm(hidden_states, self.gate_up_proj) + self.gate_up_proj_bias[..., None, :]
            logger.debug(f"{gate_up.shape=}")
            gate, up = gate_up[..., ::2], gate_up[..., 1::2]
            logger.debug(f"{gate.shape=}, {up.shape=}")

            # ゲートプロジェクションの値をクリップ
            gate = gate.clamp(min=None, max=self.limit)
            logger.debug(f"{gate.shape=}")

            # アッププロジェクションの値をクリップ
            up = up.clamp(min=-self.limit, max=self.limit)
            logger.debug(f"{up.shape=}")

            # alphaでスケーリングしたカスタムSwiGLUのゲート計算を実行
            glu = gate * torch.sigmoid(gate * self.alpha)

            # アッププロジェクションに1を足してから、ゲートを掛け合わせる（残差接続と等価）
            # そしてダウンプロジェクションを適用
            next_states = torch.bmm(((up + 1) * glu), self.down_proj)
            next_states = next_states + self.down_proj_bias[..., None, :]
            logger.debug(f"{next_states.shape=}")

            # エキスパートごとに分割
            next_states = next_states.view(num_experts, batch_size, -1, self.hidden_size)
            logger.debug(f"{next_states.shape=}")

            # ルーティング重みでエキスパートの出力を重み付け
            next_states = next_states * routing_weights.transpose(0, 1).view(num_experts, batch_size, -1)[..., None]
            logger.debug(f"{next_states.shape=}")

            # 全てのエキスパートの出力を合計
            next_states = next_states.sum(dim=0)

        logger.info(f"GptOssExpertsの順伝播完了 {next_states.shape=}, {next_states.dtype=}")
        return next_states

### GptOssTopKRouter

GptOssTopKRouterは、エキスパートにトークンを割り当てるルータネットワークの実装

In [ ]:
class GptOssTopKRouter(nn.Module):
    def __init__(self, config):
        logger.info(f"GptOssTopKRouterを初期化開始 {config.num_experts_per_tok=}, {config.num_local_experts=}, {config.hidden_size=}")
        super().__init__()

        # 上位k個のエキスパート数を設定
        # 4
        self.top_k = config.num_experts_per_tok

        # エキスパート数
        # 32
        self.num_experts = config.num_local_experts

        # 隠れ層の次元数
        # 2880
        self.hidden_dim = config.hidden_size

        # ルーティングの重みを初期化
        # (32, 2880)
        self.weight = nn.Parameter(torch.empty(self.num_experts, self.hidden_dim))

        # ルーティングのバイアスを初期化
        # (32,)
        self.bias = nn.Parameter(torch.empty(self.num_experts))
        logger.info("GptOssTopKRouterの初期化完了")

    def forward(self, hidden_states):
        logger.info(f"GptOssTopKRouterの順伝播開始 {hidden_states.shape=}, {hidden_states.dtype=}")

        # (バッチサイズ*トークン数, 隠れ層の次元数)に変形
        hidden_states = hidden_states.reshape(-1, self.hidden_dim)
        logger.debug(f"{hidden_states.shape=}")

        # ルーティングのロジットを計算
        router_logits = F.linear(hidden_states, self.weight, self.bias)  # (seq_len, num_experts)
        logger.debug(f"{router_logits.shape=}")

        # 上位k個のエキスパートを選択
        router_top_value, router_indices = torch.topk(router_logits, self.top_k, dim=-1)  # (seq_len, top_k)
        logger.debug(f"{router_top_value.shape=}, {router_indices.shape=}")

        # ソフトマックスでルーティングの重みを計算
        router_top_value = torch.nn.functional.softmax(router_top_value, dim=1, dtype=router_top_value.dtype)
        logger.debug(f"{router_top_value.shape=}")

        # スパースなルーティングスコアを作成
        router_scores = torch.zeros_like(router_logits).scatter_(1, router_indices, router_top_value)
        logger.debug(f"{router_scores.shape=}")

        logger.info(f"GptOssTopKRouterの順伝播完了 {router_scores.shape=}, {router_scores.dtype=}, {router_indices.shape=}, {router_indices.dtype=}")
        return router_scores, router_indices

### GptOssMLP

GptOssMLPは、MoE層の実装

In [ ]:
# MegaBlocksMoeMLPカーネルを使用（https://huggingface.co/docs/kernels/main/en/index）
@use_kernel_forward_from_hub("MegaBlocksMoeMLP")
class GptOssMLP(nn.Module):
    def __init__(self, config):
        logger.info("GptOssMLPを初期化開始")
        super().__init__()

        # ルーティングネットワークを初期化
        self.router = GptOssTopKRouter(config)

        # 複数のエキスパートを初期化
        self.experts = GptOssExperts(config)
        logger.info("GptOssMLPの初期化完了")

    def forward(self, hidden_states):
        logger.info(f"GptOssMLPの順伝播開始 {hidden_states.shape=}, {hidden_states.dtype=}")

        # ルーティングスコアとインデックスを取得
        router_scores, router_indices = self.router(hidden_states)  # (num_experts, seq_len)

        # エキスパートにルーティングして出力を取得
        routed_out = self.experts(hidden_states, router_indices=router_indices, routing_weights=router_scores)

        logger.info(f"GptOssMLPの順伝播完了 {routed_out.shape=}, {routed_out.dtype=}")
        return routed_out, router_scores

### GptOssRotaryEmbedding

GptOssRoutaryEmbeddingは、RoPE（Rotary Position Embedding, 回転位置埋め込み）の計算に必要なsinとcosを計算するクラス

In [ ]:
class GptOssRotaryEmbedding(nn.Module):

    # 逆周波数
    inv_freq: torch.Tensor  # fix linting for `register_buffer`

    def __init__(self, config: GptOssConfig, device=None):
        logger.info(f"GptOssRotaryEmbeddingを初期化開始 {config.max_position_embeddings=} {config.rope_scaling=}")

        super().__init__()
        # BC: "rope_type" was originally "type"
        if hasattr(config, "rope_scaling") and isinstance(config.rope_scaling, dict):
            self.rope_type = config.rope_scaling.get("rope_type", config.rope_scaling.get("type"))
        else:
            self.rope_type = "default"

        # 131072
        self.max_seq_len_cached = config.max_position_embeddings

        # 131072
        self.original_max_seq_len = config.max_position_embeddings

        self.config = config

        # _compute_yarn_parameters
        self.rope_init_fn = ROPE_INIT_FUNCTIONS[self.rope_type]
        logger.info(f"RoPEの初期化関数: {self.rope_init_fn.__name__}")

        # 32, 1.3466
        inv_freq, self.attention_scaling = self.rope_init_fn(self.config, device)
        logger.info(f"逆周波数の計算 {inv_freq.shape=}, {inv_freq.dtype=}, {self.attention_scaling=}")

        self.register_buffer("inv_freq", inv_freq, persistent=False)

        self.original_inv_freq = self.inv_freq
        logger.info("GptOssRotaryEmbeddingの初期化完了")

    @torch.no_grad()
    @dynamic_rope_update  # power user: used with advanced RoPE types (e.g. dynamic rope)
    def forward(self, x, position_ids):
        logger.info(f"GptOssRotaryEmbeddingの順伝播開始 {x.shape=}, {x.dtype=}, {position_ids.shape=}, {position_ids.dtype=}")

        # 逆周波数を拡張
        # (32,) -> (1, 32, 1) -> (1, 32, 1)
        # float32
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1).to(x.device)
        logger.debug(f"{inv_freq_expanded.shape=}, {inv_freq_expanded.dtype=}")

        # 位置IDを拡張
        # (1, 69) -> (1, 1, 69)
        # int64 -> float32
        position_ids_expanded = position_ids[:, None, :].float()
        logger.debug(f"{position_ids_expanded.shape=}, {position_ids_expanded.dtype=}")

        device_type = x.device.type if isinstance(x.device.type, str) and x.device.type != "mps" else "cpu"
        with torch.autocast(device_type=device_type, enabled=False):  # Force float32
            # アップキャストして角度を計算（周波数 * 位置）
            # (1, 32, 1) @ (1, 1, 69) -> (1, 32, 69) -> (1, 69, 32)
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = freqs

            # コサインを計算し、スケーリングを適用
            # (1, 69, 32)
            cos = emb.cos() * self.attention_scaling

            # サインを計算し、スケーリングを適用
            # (1, 69, 32)
            sin = emb.sin() * self.attention_scaling

        logger.info(f"GptOssRotaryEmbeddingの順伝播完了 {cos.shape=}, {cos.dtype=}, {sin.shape=}, {sin.dtype=}")
        return cos.to(x.dtype), sin.to(x.dtype)

In [ ]:
def _apply_rotary_emb(
    x: torch.Tensor,
    cos: torch.Tensor,
    sin: torch.Tensor,
) -> torch.Tensor:
    logger.info(f"_apply_rotary_embの順伝播開始 {x.shape=}, {x.dtype=}, {cos.shape=}, {cos.dtype=}, {sin.shape=}, {sin.dtype=}")

    first_half, second_half = torch.chunk(x, 2, dim=-1)
    logger.debug(f"{first_half.shape=}, {second_half.shape=}")

    first_ = first_half * cos - second_half * sin
    logger.debug(f"{first_.shape=}")

    second_ = second_half * cos + first_half * sin
    logger.debug(f"{second_.shape=}")

    res = torch.cat((first_, second_), dim=-1)
    logger.info(f"_apply_rotary_embの順伝播完了 {res.shape=}, {res.dtype=}")
    return res

In [ ]:
def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    logger.info(f"apply_rotary_pos_embの順伝播開始 {q.shape=}, {q.dtype=}, {k.shape=}, {k.dtype=}, {cos.shape=}, {cos.dtype=}, {sin.shape=}, {sin.dtype=}, {position_ids=}, {unsqueeze_dim=}")

    cos = cos.unsqueeze(unsqueeze_dim)
    logger.debug(f"{cos.shape=}")

    sin = sin.unsqueeze(unsqueeze_dim)
    logger.debug(f"{sin.shape=}")

    q_embed = _apply_rotary_emb(q, cos, sin)

    k_embed = _apply_rotary_emb(k, cos, sin)

    logger.info(f"apply_rotary_pos_embの順伝播完了 {q_embed.shape=}, {q_embed.dtype=}, {k_embed.shape=}, {k_embed.dtype=}")
    return q_embed, k_embed

### GptOssAttention

GptOssAttentionは、アテンションシンクを持つマルチヘッドセルフアテンションの実装

repeat_kvは、GQA（Grouped Query Attention）用にキーとバリューを複製する関数

In [ ]:
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    logger.info(f"repeat_kvの開始 {hidden_states.shape=}, {hidden_states.dtype=}, {n_rep=}")
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states

    # (1, 8, 69, 64) -> (1, 8, 8, 69, 64), bfloat16
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    logger.debug(f"{hidden_states.shape=}, {hidden_states.dtype=}")

    # (1, 8, 8, 69, 64) -> (1, 64, 69, 64), bfloat16
    res = hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)
    logger.info(f"repeat_kvの完了 {res.shape=}, {res.dtype=}")
    return res

eager_attention_forwardは、アテンションシンクを実装したeagerな（原著論文に忠実で非効率な）実装

In [ ]:
def eager_attention_forward(
    module: nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    scaling: float,
    dropout: float = 0.0,
    **kwargs,
):
    logger.info(f"eager_attention_forwardの開始 {query.shape=}, {key.shape=}, {value.shape=}, {scaling=}, {dropout=} {attention_mask.shape=}")

    # GQA用にキーを複製
    # (1, 8, 69, 64) -> (1, 64, 69, 64)
    key_states = repeat_kv(key, module.num_key_value_groups)
    logger.debug(f"キーを複製 {key_states.shape=}, {key_states.dtype=}")

    # GQA用にバリューを複製
    # (1, 8, 69, 64) -> (1, 64, 69, 64)
    value_states = repeat_kv(value, module.num_key_value_groups)
    logger.debug(f"バリューを複製 {value_states.shape=}, {value_states.dtype=}")

    # アテンションロジットを計算
    # QK^T / sqrt(d_k) 
    # (1, 64, 69, 64) @ (1, 64, 64, 69) -> (1, 64, 69, 69)
    attn_weights = torch.matmul(query, key_states.transpose(2, 3)) * scaling
    logger.debug(f"アテンションロジットを計算 {attn_weights.shape=}, {attn_weights.dtype=}")

    # 因果マスクを適用
    if attention_mask is not None:
        # (1, 1, 69, 69) -> (1, 1, 69, 69)
        causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
        logger.debug(f"{causal_mask.shape=}, {causal_mask.dtype=}")
        # (1, 64, 69, 69) + (1, 1, 69, 69) -> (1, 64, 69, 69), bfloat16
        attn_weights = attn_weights + causal_mask
        logger.debug(f"因果マスク適用 {attn_weights.shape=}, {attn_weights.dtype=}")

    # アテンションシンクを作成 
    # (1, 64, 69, 1), bfloat16
    sinks = module.sinks.reshape(1, -1, 1, 1).expand(query.shape[0], -1, query.shape[-2], -1)
    logger.debug(f"{sinks.shape=}, {sinks.dtype=}")

    # アテンションロジットにアテンションシンクを追加
    # (1, 64, 69, 69 + 1) -> (1, 64, 69, 70), bfloat16
    combined_logits = torch.cat([attn_weights, sinks], dim=-1)
    logger.debug(f"シンクを追加 {combined_logits.shape=}, {combined_logits.dtype=}")

    # 安定化のために最大値を減算
    # (1, 65, 69, 70) - (1, 65, 69, 1) -> (1, 65, 69, 70), bfloat16
    # This was not in the original implementation and slightly affect results; it prevents overflow in BF16/FP16
    # when training with bsz>1 we clamp max values.
    combined_logits = combined_logits - combined_logits.max(dim=-1, keepdim=True).values
    logger.debug(f"最大値を減算 {combined_logits.shape=}, {combined_logits.dtype=}")

    # アテンションスコアを計算
    # (1, 64, 69, 70) -> (1, 64, 69, 70), bfloat16
    probs = F.softmax(combined_logits, dim=-1, dtype=combined_logits.dtype)
    logger.debug(f"アテンションスコアを計算 {probs.shape=}, {probs.dtype=}")

    # シンクの確率を除去
    # (1, 64, 69, 70) -> (1, 64, 69, 69), bfloat16
    scores = probs[..., :-1]  # we drop the sink here
    logger.debug(f"シンクの確率を除去 {scores.shape=}, {scores.dtype=}")

    # ドロップアウトを適用
    # (1, 64, 69, 69), bfloat16
    attn_weights = nn.functional.dropout(scores, p=dropout, training=module.training)

    # アテンションを計算
    # (1, 64, 69, 69) @ (1, 64, 69, 64) -> (1, 64, 69, 64)
    attn_output = torch.matmul(attn_weights, value_states)
    logger.debug(f"アテンション {attn_output.shape=}, {attn_output.dtype=}")

    # (1, 64, 69, 64) -> (1, 69, 64, 64)
    attn_output = attn_output.transpose(1, 2).contiguous()
    logger.debug(f"アテンションを転置 {attn_output.shape=}, {attn_output.dtype=}")

    # (1, 69, 64, 64), bfloat16
    logger.info(f"eager_attention_forwardの完了 {attn_output.shape=}, {attn_output.dtype=}")
    return attn_output, attn_weights

In [ ]:
class GptOssAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: GptOssConfig, layer_idx: int):
        logger.info(f"GptOssAttentionを初期化開始 {config.hidden_size=}, {config.num_attention_heads=}, {config.num_key_value_heads=}, {config.attention_bias=}, {config.attention_dropout=}, {layer_idx=}")

        super().__init__()
        self.config = config

        self.layer_idx = layer_idx

        self.head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
        logger.debug(f"{self.head_dim=}")

        self.num_key_value_groups = config.num_attention_heads // config.num_key_value_heads
        logger.debug(f"{self.num_key_value_groups=}")

        self.scaling = self.head_dim**-0.5
        logger.debug(f"{self.scaling=}")

        self.attention_dropout = config.attention_dropout
        logger.debug(f"{self.attention_dropout=}")
        
        self.is_causal = True

        # クエリのプロジェクション
        # 2880 -> 64 * 64
        self.q_proj = nn.Linear(
            config.hidden_size, config.num_attention_heads * self.head_dim, bias=config.attention_bias
        )

        # キーのプロジェクション
        self.k_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )

        # バリューのプロジェクション
        self.v_proj = nn.Linear(
            config.hidden_size, config.num_key_value_heads * self.head_dim, bias=config.attention_bias
        )

        # 出力のプロジェクション
        self.o_proj = nn.Linear(
            config.num_attention_heads * self.head_dim, config.hidden_size, bias=config.attention_bias
        )

        # スライディングウィンドウの設定
        self.sliding_window = config.sliding_window if config.layer_types[layer_idx] == "sliding_attention" else None
        logger.debug(f"{self.sliding_window=}")

        # アテンションシンクの初期化
        self.sinks = nn.Parameter(torch.empty(config.num_attention_heads))

        logger.info("GptOssAttentionの初期化完了")

    @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor],
        past_key_values: Optional[Cache] = None,
        cache_position: Optional[torch.LongTensor] = None,
        **kwargs: Unpack[TransformersKwargs],
    ) -> tuple[torch.Tensor, torch.Tensor]:
        logger.info(f"GptOssAttentionの順伝播開始 {hidden_states.shape=}, {position_embeddings[0].shape=}, {position_embeddings[1].shape=}, {past_key_values=}, {cache_position.shape=}, {attention_mask.shape=}")

        # (1, 69, 2880) -> (1, 69)
        input_shape = hidden_states.shape[:-1]
        logger.debug(f"{input_shape=}")

        # (1, 69, -1, 64)
        hidden_shape = (*input_shape, -1, self.head_dim)
        logger.debug(f"{hidden_shape=}")

        # クエリを計算
        # (1, 69, 2880) -> (1, 69, 4096) -> (1, 69, 64, 64) -> (1, 64, 69, 64)
        query_states = self.q_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        logger.debug(f"クエリを計算 {query_states.shape=}")

        # キーを計算
        # (1, 69, 2880) -> (1, 69, 4096) -> (1, 69, 64, 64) -> (1, 64, 69, 64)
        key_states = self.k_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        logger.debug(f"キーを計算 {key_states.shape=}")

        # バリューを計算
        # (1, 69, 2880) -> (1, 69, 4096) -> (1, 69, 64, 64) -> (1, 64, 69, 64)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)
        logger.debug(f"バリューを計算 {value_states.shape=}")

        # cosとsinを取得
        # (1, 69, 32), (1, 69, 32)
        cos, sin = position_embeddings
        logger.debug(f"RoPEのcosとsinを取得 {cos.shape=}, {sin.shape=}")

        # RoPEを適用
        # (1, 64, 69, 64), (1, 64, 69, 64)
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)
        logger.debug(f"RoPEを適用 {query_states.shape=}, {key_states.shape=}")

        # キーバリューのキャッシュを更新
        if past_key_values is not None:
            cache_kwargs = {"cache_position": cache_position}
            # (1, 8, 69, 64), (1, 8, 69, 64)
            key_states, value_states = past_key_values.update(key_states, value_states, self.layer_idx, cache_kwargs)
            logger.debug(f"キーバリューキャッシュを更新 {key_states.shape=}, {value_states.shape=}")

        # アテンションの実装を選択
        attention_interface: Callable = eager_attention_forward
        if self.config._attn_implementation != "eager":
            attention_interface = ALL_ATTENTION_FUNCTIONS[self.config._attn_implementation]

        # eager_attention_forward
        logger.debug(f"使用するアテンション実装: {attention_interface.__name__}")

        # アテンションを計算
        # (1, 64, 69, 64), (1, 64, 69, 64), (1, 64, 69, 64) -> (1, 69, 64, 64), None
        attn_output, attn_weights = attention_interface(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0 if not self.training else self.attention_dropout,
            scaling=self.scaling,
            sliding_window=self.sliding_window,
            s_aux=self.sinks,  # diff with Llama
            **kwargs,
        )
        logger.debug(f"アテンション {attn_output.shape=}")

        # 出力を元の形状に戻す
        # (1, 69, 64, 64) -> (1, 69, 4096)
        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        logger.debug(f"アテンションを元の形状に戻す {attn_output.shape=}")

        # 出力プロジェクションを適用
        # (1, 69, 4096) -> (1, 69, 2880)
        attn_output = self.o_proj(attn_output)

        logger.info(f"GptOssAttentionの順伝播完了 {attn_output.shape=}, {attn_output.dtype=}, {attn_weights.shape=}, {attn_weights.dtype=}")
        return attn_output, attn_weights

### GptOssDecoderLayer

GptOssDecoderLayerは、gpt-ossモデルの単一のTransformerのデコーダー層

Pre-LNを採用し、アテンションとMoE層の前にレイヤー正規化層が実装されている

In [ ]:
class GptOssDecoderLayer(GradientCheckpointingLayer):
    def __init__(self, config: GptOssConfig, layer_idx: int):
        logger.info(f"GptOssDecoderLayerを初期化開始 {config.hidden_size=}, {config.intermediate_size=}, {config.num_attention_heads=}, {config.num_key_value_heads=}, {config.attention_bias=}, {config.attention_dropout=}, {layer_idx=}")

        super().__init__()

        # 2880
        self.hidden_size = config.hidden_size

        # セルフアテンション
        self.self_attn = GptOssAttention(config=config, layer_idx=layer_idx)

        # MoE層
        self.mlp = GptOssMLP(config)

        # 2つのレイヤー正規化層
        # 2880
        self.input_layernorm = GptOssRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = GptOssRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

        # スライディングアテンションとフルアテンションが交互に選択
        self.attention_type = config.layer_types[layer_idx]
        logger.debug(f"{self.attention_type=}")

        logger.info("GptOssDecoderLayerの初期化完了")

    @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        use_cache: Optional[bool] = False,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,  # necessary, but kept here for BC
        **kwargs: Unpack[TransformersKwargs],
    ) -> torch.Tensor:

        logger.info(f"GptOssDecoderLayerの順伝播開始 {hidden_states.shape=}, {hidden_states.dtype=}, {position_ids.shape=}, {past_key_values=}, {use_cache=}, {cache_position.shape=}, {position_embeddings[0].shape=}, {position_embeddings[1].shape=}, {attention_mask.shape=}")

        # 残差接続用
        # (1, 69, 2880)
        residual = hidden_states

        # レイヤー正規化を適用（Pre-LN）
        # (1, 69, 2880) -> (1, 69, 2880)
        hidden_states = self.input_layernorm(hidden_states)

        # セルフアテンションを適用
        # (1, 69, 2880) -> (1, 69, 2880)
        hidden_states, _ = self.self_attn(
            hidden_states=hidden_states, # (1, 69, 2880)
            attention_mask=attention_mask, # (1, 1, 69, 69)
            position_ids=position_ids, # (1, 69)
            past_key_values=past_key_values,
            use_cache=use_cache, # True
            cache_position=cache_position, # (69,)
            position_embeddings=position_embeddings, # ((1, 69, 32), (1, 69, 32))
            **kwargs,
        )

        # 残差接続を適用
        # (1, 69, 2880) + (1, 69, 2880) -> (1, 69, 2880)
        hidden_states = residual + hidden_states

        # 残差接続用
        # (1, 69, 2880)
        residual = hidden_states

        # レイヤー正規化を適用（Pre-LN）
        # (1, 69, 2880) -> (1, 69, 2880)
        hidden_states = self.post_attention_layernorm(hidden_states)

        # MoE層を適用
        # (1, 69, 2880) -> (1, 69, 2880)
        logger.debug(f"MoE層の入力 {hidden_states.shape=}, {hidden_states.dtype=}")
        hidden_states, _ = self.mlp(hidden_states)  # diff with llama: router scores
        logger.debug(f"MoE層の出力 {hidden_states.shape=}, {hidden_states.dtype=}")

        # 残差接続を適用
        # (1, 69, 2880) + (1, 69, 2880) -> (1, 69, 2880)
        hidden_states = residual + hidden_states

        return hidden_states

### GptOssPreTrainedModel

GptOssPreTrainedModelは、gpt-ossの事前学習済みモデルの設定クラス

In [ ]:
class GptOssPreTrainedModel(PreTrainedModel):
    config: GptOssConfig
    base_model_prefix = "model"
    supports_gradient_checkpointing = True
    _no_split_modules = ["GptOssDecoderLayer"]
    _skip_keys_device_placement = ["past_key_values"]
    _supports_flash_attn = True
    _supports_sdpa = False
    _supports_flex_attn = True

    _can_compile_fullgraph = True
    _supports_attention_backend = True
    _can_record_outputs = {
        "router_logits": OutputRecorder(GptOssTopKRouter, index=0),
        "hidden_states": GptOssDecoderLayer,
        "attentions": GptOssAttention,
    }
    _keep_in_fp32_modules = ["post_attention_layernorm", "input_layernorm", "norm"]
    _supports_flash_attention = False
    _supports_flex_attention = False

    def _init_weights(self, module):
        std = self.config.initializer_range
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Parameter):
            module.data.normal_(mean=0.0, std=std)
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, GptOssRMSNorm):
            module.weight.data.fill_(1.0)
        elif isinstance(module, GptOssExperts):
            module.gate_up_proj.data.normal_(mean=0.0, std=std)
            module.gate_up_proj_bias.data.zero_()
            module.down_proj.data.normal_(mean=0.0, std=std)
            module.down_proj_bias.data.zero_()
        elif isinstance(module, GptOssAttention):
            module.sinks.data.normal_(mean=0.0, std=std)
        elif isinstance(module, GptOssTopKRouter):
            module.weight.data.normal_(mean=0.0, std=std)
            module.bias.data.normal_(mean=0.0, std=std)

### GptOssModel

GptOssModelは、複数のGptOssDecoderLayerを重ねたベースクラス

In [ ]:
class GptOssModel(GptOssPreTrainedModel):
    _no_split_modules = ["GptOssDecoderLayer"]

    def __init__(self, config: GptOssConfig):
        logger.info(f"GptOssModelを初期化開始 {config.vocab_size=}, {config.hidden_size=}, {config.num_hidden_layers=}, {config.pad_token_id=}")
        super().__init__(config)

        # 199999
        self.padding_idx = config.pad_token_id

        # 201088
        self.vocab_size = config.vocab_size

        # 入力の埋め込み層
        # 202112 -> 2880
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)

        # デコーダー層をスタック
        # 24層
        self.layers = nn.ModuleList(
            [GptOssDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )

        # 最終レイヤー正規化層
        # 2880
        self.norm = GptOssRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

        # RoPEのsinとcosの計算モジュール
        self.rotary_emb = GptOssRotaryEmbedding(config=config)

        self.gradient_checkpointing = False

        # Initialize weights and apply final processing
        self.post_init()
        logger.info("GptOssModelの初期化完了")

    @check_model_inputs
    @auto_docstring
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        **kwargs: Unpack[TransformersKwargs],
    ) -> MoeModelOutputWithPast:
        logger.info(f"GptOssModelの順伝播開始 {input_ids.shape=}, {position_ids.shape=}, {past_key_values=}, {inputs_embeds=}, {use_cache=}, {cache_position.shape=}, {attention_mask.shape=}")

        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError("You must specify exactly one of input_ids or inputs_embeds")

        if use_cache and past_key_values is None:
            past_key_values = DynamicCache(config=self.config)
            logger.info("DynamicCacheを初期化")

        if inputs_embeds is None:
            # (1, 69) -> (1, 69, 2880)
            inputs_embeds = self.embed_tokens(input_ids)
            logger.debug(f"入力の埋め込みを計算 {inputs_embeds.shape=}, {inputs_embeds.dtype=}")

        if cache_position is None:
            past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
            cache_position = torch.arange(
                past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1], device=inputs_embeds.device
            )
            logger.info(f"cache_positionを計算 {cache_position.shape=}, {cache_position.dtype=}")

        if position_ids is None:
            position_ids = cache_position.unsqueeze(0)
            logger.info(f"position_idsを計算 {position_ids.shape=}, {position_ids.dtype=}")

        # It may already have been prepared by e.g. `generate`
        if not isinstance(causal_mask_mapping := attention_mask, dict):
            mask_kwargs = {
                "config": self.config,
                "input_embeds": inputs_embeds,
                "attention_mask": attention_mask,
                "cache_position": cache_position,
                "past_key_values": past_key_values,
            }
            causal_mask_mapping = {
                "full_attention": create_causal_mask(**mask_kwargs), # (1, 1, 69, 69)
                "sliding_attention": create_sliding_window_causal_mask(**mask_kwargs), # (1, 1, 69, 69)
            }
            logger.debug(f"因果マスクを準備 {causal_mask_mapping['full_attention'].shape=}, {causal_mask_mapping['sliding_attention'].shape=}")

        hidden_states = inputs_embeds

        # RoPEのsinとcosを計算（すべての層で共有）
        # (1, 1, 69), (1, 1, 69)
        position_embeddings = self.rotary_emb(hidden_states, position_ids)
        logger.debug(f"{position_embeddings[0].shape=}, {position_embeddings[1].shape=}")

        # デコーダー層を順伝播
        # (1, 69, 2880) -> (1, 69, 2880)
        for decoder_layer in self.layers:
            hidden_states = decoder_layer(
                hidden_states, # (1, 69, 2880)
                attention_mask=causal_mask_mapping[decoder_layer.attention_type], # (1, 1, 69, 69)
                position_ids=position_ids, # (1, 69)
                past_key_values=past_key_values,
                use_cache=use_cache, # True
                cache_position=cache_position, # (69,)
                position_embeddings=position_embeddings, # ((1, 1, 69), (1, 1, 69))
                **kwargs,
            )
        
        # 最終レイヤー正規化を適用
        # (1, 69, 2880) -> (1, 69, 2880)
        hidden_states = self.norm(hidden_states)

        logger.info(f"GptOssModelの順伝播完了 {hidden_states.shape=}, {hidden_states.dtype=}")
        return MoeModelOutputWithPast(
            last_hidden_state=hidden_states,
            past_key_values=past_key_values,
        )

### GptOssForCausalLM

In [ ]:
def load_balancing_loss_func(
    gate_logits: Union[torch.Tensor, tuple[torch.Tensor], None],
    num_experts: Optional[int] = None,
    top_k=2,
    attention_mask: Optional[torch.Tensor] = None,
) -> Union[torch.Tensor, int]:
    r"""
    Computes auxiliary load balancing loss as in Switch Transformer - implemented in Pytorch.

    See Switch Transformer (https://huggingface.co/papers/2101.03961) for more details. This function implements the loss
    function presented in equations (4) - (6) of the paper. It aims at penalizing cases where the routing between
    experts is too unbalanced.

    Args:
        gate_logits:
            Logits from the `gate`, should be a tuple of model.config.num_hidden_layers tensors of
            shape [batch_size X sequence_length, num_experts].
        num_experts:
            Number of experts
        top_k:
            The number of experts to route per-token, can be also interpreted as the `top-k` routing
            parameter.
        attention_mask (`torch.Tensor`, *optional*):
            The attention_mask used in forward function
            shape [batch_size X sequence_length] if not None.

    Returns:
        The auxiliary loss.
    """
    if gate_logits is None or not isinstance(gate_logits, tuple):
        return 0

    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        concatenated_gate_logits = torch.cat([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)

    routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)

    _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)

    expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts)

    if attention_mask is None:
        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.mean(expert_mask.float(), dim=0)

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.mean(routing_weights, dim=0)
    else:
        batch_size, sequence_length = attention_mask.shape
        num_hidden_layers = concatenated_gate_logits.shape[0] // (batch_size * sequence_length)

        # Compute the mask that masks all padding tokens as 0 with the same shape of expert_mask
        expert_attention_mask = (
            attention_mask[None, :, :, None, None]
            .expand((num_hidden_layers, batch_size, sequence_length, top_k, num_experts))
            .reshape(-1, top_k, num_experts)
            .to(compute_device)
        )

        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.sum(expert_mask.float() * expert_attention_mask, dim=0) / torch.sum(
            expert_attention_mask, dim=0
        )

        # Compute the mask that masks all padding tokens as 0 with the same shape of tokens_per_expert
        router_per_expert_attention_mask = (
            attention_mask[None, :, :, None]
            .expand((num_hidden_layers, batch_size, sequence_length, num_experts))
            .reshape(-1, num_experts)
            .to(compute_device)
        )

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.sum(routing_weights * router_per_expert_attention_mask, dim=0) / torch.sum(
            router_per_expert_attention_mask, dim=0
        )

    overall_loss = torch.sum(tokens_per_expert * router_prob_per_expert.unsqueeze(0))
    return overall_loss * num_experts

In [ ]:
class GptOssForCausalLM(GptOssPreTrainedModel, GenerationMixin):
    _tied_weights_keys = ["lm_head.weight"]
    _tp_plan = {"lm_head": "colwise_rep"}
    _pp_plan = {"lm_head": (["hidden_states"], ["logits"])}

    def __init__(self, config):
        logger.info(f"GptOssForCausalLMを初期化開始 {config.vocab_size=}, {config.hidden_size=}, {config.num_hidden_layers=}, {config.pad_token_id=}, {config.router_aux_loss_coef=}, {config.num_local_experts=}, {config.num_experts_per_tok=}")

        super().__init__(config)

        self.model = GptOssModel(config)

        # 201088
        self.vocab_size = config.vocab_size

        # 2880 -> 201088
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # 0.9
        self.router_aux_loss_coef = config.router_aux_loss_coef

        # 32
        self.num_experts = config.num_local_experts

        # 4
        self.num_experts_per_tok = config.num_experts_per_tok

        # Initialize weights and apply final processing
        self.post_init()
        logger.info("GptOssForCausalLMの初期化完了")

    @can_return_tuple
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_router_logits: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        logits_to_keep: Union[int, torch.Tensor] = 0,
        **kwargs: Unpack[TransformersKwargs],
    ) -> MoeCausalLMOutputWithPast:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
            (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Example:

        ```python
        >>> from transformers import AutoTokenizer, GptOssForCausalLM

        >>> model = GptOssForCausalLM.from_pretrained("mistralai/GptOss-8x7B-v0.1")
        >>> tokenizer = AutoTokenizer.from_pretrained("mistralai/GptOss-8x7B-v0.1")

        >>> prompt = "Hey, are you conscious? Can you talk to me?"
        >>> inputs = tokenizer(prompt, return_tensors="pt")

        >>> # Generate
        >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
        >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        "Hey, are you conscious? Can you talk to me?\nI'm not conscious, but I can talk to you."
        ```"""
        logger.info(f"GptOssForCausalLMの順伝播開始 {input_ids.shape=}, {position_ids.shape=}, {past_key_values=}, {use_cache=}, {cache_position.shape=}, {logits_to_keep=}, {attention_mask.shape=}")

        # False（ルーターのロジットを出力しない）
        output_router_logits = (
            output_router_logits if output_router_logits is not None else self.config.output_router_logits
        )
        logger.debug(f"{output_router_logits=}")

        # デコーダーを順伝播
        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs: MoeModelOutputWithPast = self.model(
            input_ids=input_ids, # (1, 69)
            attention_mask=attention_mask, # (1, 69)
            position_ids=position_ids, # (1, 69)
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds, # None
            use_cache=use_cache, # True
            output_router_logits=output_router_logits, # False
            cache_position=cache_position, # (69,)
            **kwargs,
        )

        hidden_states = outputs.last_hidden_state
        logger.debug(f"最終隠れ状態を取得 {hidden_states.shape=}, {hidden_states.dtype=}")

        # Only compute necessary logits, and do not upcast them to float if we are not computing the loss
        slice_indices = slice(-logits_to_keep, None) if isinstance(logits_to_keep, int) else logits_to_keep
        logger.debug(f"スライスインデックスを計算 {slice_indices=}")

        logits = self.lm_head(hidden_states[:, slice_indices, :])
        logger.debug(f"ロジットを計算 {logits.shape=}, {logits.dtype=}")

        # 訓練時は損失を計算
        loss = None
        if labels is not None:
            loss = self.loss_function(logits, labels, self.vocab_size, **kwargs)

        # ルーターの補助損失を計算
        aux_loss = None
        if output_router_logits:
            aux_loss = load_balancing_loss_func(
                outputs.router_logits,
                self.num_experts,
                self.num_experts_per_tok,
                attention_mask,
            )
            if labels is not None:
                loss += self.router_aux_loss_coef * aux_loss.to(loss.device)  # make sure to reside in the same device

        return MoeCausalLMOutputWithPast(
            loss=loss,
            aux_loss=aux_loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            router_logits=outputs.router_logits,
        )

### 1トークンのみ推論

In [ ]:
from transformers import AutoTokenizer

model_id = "openai/gpt-oss-20b"

# トークナイザーを読み込み
# 実装はPreTrainedTokenizerFast
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer

In [ ]:
# パラメータを読み込み
# モデルの重みはMXFP4形式
# そのまま読み込むとVRAMを15GB以下に節約できる
# bfloat16で読み込むとVRAMが48GB必要になる
# FlashAttention3を有効にするとより高速化が可能

model = GptOssForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype="auto",
    # attn_implementation="kernels-community/vllm-flash-attn3",
)
model

In [ ]:
messages = [{"role": "user", "content": "Hello!"}]
messages

In [ ]:
raw_inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    reasoning_effort="medium",
    return_tensors="pt",
    tokenize=False,
)
raw_inputs

In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    reasoning_effort="medium",
    return_tensors="pt",
    tokenize=True,
    return_dict=True,
)
inputs, inputs["input_ids"].shape

In [ ]:
generated = model.generate(**inputs, max_new_tokens=1)
generated

In [ ]:
tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:])

In [ ]:
tokenizer.decode(generated[0])

### ログをオフにして推論

In [ ]:
logger.setLevel(logging.ERROR)

messages = [
    {"role": "user", "content": "Tell me recommended tourist attractions in Fukuoka city."},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    reasoning_effort="medium",
    return_tensors="pt",
    tokenize=True,
    return_dict=True,
)

generated = model.generate(**inputs, max_new_tokens=4096)

tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:])

The user wants recommended tourist attractions in Fukuoka City. Provide a list, grouping by type. Provide brief explanation, practical tips, and maybe mention accessibility, operation hours, admission, best times, etc. Also could mention off-the-beaten path attractions, local experience, food spots, etc. Should be tailored, maybe mention some must-visit highlights: Fukuoka Castle ruins, Ohori Park, Fukuoka Tower, Canal City, Tenjin, Nakasu, Hakata Shrine, Momochi Seaside Park, etc. Might also suggest some local experiences like yatai stalls, ramen stalls, maybe some cultural experience. The user didn't provide language preference, but we output in English likely. Provide bullet points. If the user wants more detailed? But just "recommended tourist attractions" is asked. Let's provide a list with brief descriptions and tips. Also include a short note on transportation. Provide also a suggestion for a day itinerary? Possibly optional. Could mention a recommended day or two itinerary.

Let's craft the response: structured with categories: historical sites, parks & nature, modern attractions, shopping & nightlife, local experience. Provide key details: how to get there, admission, times, best times, recommended activities. Also mention things like "don't miss" for each. Provide in a concise but thorough manner, maybe 10-12 highlights.

Let's proceed.

## Highlights of Fukuoka City – A Quick‑Guide for Visitors  

Below is a curated list of the most iconic, off‑beat, and “must‑taste” experiences in Fukuoka.  I’ve grouped them by theme and thrown in practical details (transport, opening hours, tips) so you can plan without fuss.

| **Theme** | **Attraction** | **Why It’s Worth It** | **Key Practical Notes** |
|-----------|----------------|----------------------|------------------------|
| **Historic & Cultural** | **Fukuoka Castle Ruins (Maizuru‑tō)** | The only surviving part of an old castle – great for sweeping city views. | 10 am–6 pm, free entry. Open all year. |
| | **Hakata Shrine (Hakata Jinja)** | One of Fukuoka’s three major shrines; a quiet spot to experience Shinto rituals. | 6 am–5 pm, free. Wear respectful clothing when entering. |
| | **Nokonoshima Island (Kokura‑Jima)** | An 18‑hectare island with botanical gardens, ponds, and a lighthouse. | Bus + ferry, 7 am–10 pm. Great for a day trip. |
| **Parks & Nature** | **Ohori Park** | Classic Japanese strolling pond – ideal for a leisurely walk or paddle boat. | 7 am–10 pm, free. Great spot for lunch around the tea house. |
| | **Momochi Seaside Park** | Sand + sea + modern architecture (Fukuoka Tower). | 24 hr, free. Check sunset times for best photos. |
| | **Fukuoka City Zoo & Botanical Garden** | Family‑friendly with diverse animal species and a huge butterfly house. | 10 am–5 pm, ¥550 (adult). |
| **Contemporary & Entertainment** | **Fukuoka Tower** | Highest building in Kyushu – offers panoramic views and an observation deck. | 10 am–9 pm, ¥590. Visit mid‑afternoon for sunset views. |
| | **Canal City Hakata** | Shopping, dining, and a “river” inside a mall. Great for street‑style photographers. | 10 am–10 pm. Free to wander, but some attractions cost extra. |
| | **Daimyo‑sanjō** (Dazaifu) | Though technically outside Fukuoka City, it’s a 30‑minute train ride and the city’s spiritual heart. | 9 am–7 pm, free (except for some temples). |
| **Shopping & Nightlife** | **Tenjin** (central business district) | The hub for fashion, electronics and underground shopping. | 10 am–9 pm (stores). |
| | **Nakasu** (red‑light district) | Best for yatai (street food stalls), izakayas, and nightlife. | Evening to midnight. |
| | **Hakata Station** | Iconic underground shopping and a great place for ramen. | 7 am–9 pm (shops). |
| **Local Experience** | **Yatai Stalls** | Street food tents selling ramen, yakitori, gyoza, etc. | Late afternoon to midnight. Great for trying local cuisine. |
| | **Matsusaka‑tai (Japanese horse mackerel) at **Matsuzaka‑Taiya** | Fresh seafood at a local market. | Early‑morning. |
| | **Fukuoka Asian Art Museum** | Focus on contemporary Asian art. | 10 am–5 pm; free entry. |

---

### Suggested Mini‑Itineraries

| **Duration** | **Day 1** | **Day 2** |
|--------------|-----------|-----------|
| **1 Day** | 1. Morning: Ohori Park & Fukuoka Tower (take photos of the city from the top).  <br>2. Lunch at Yatai (try Hakata ramen).  <br>3. Afternoon: Tenjin shopping & Canal City.  <br>4. Evening: Nakasu yatai and a stroll along the canal. | 1. Quick visit to Hakata Shrine in the morning.  <br>2. Lunch at Hakata Station (classic Hakata ramen).  <br>3. Explore Kokura‑Jima (or Momochi Seaside).  <br>4. Evening: Walk around the Fukuoka City Zoo or dine in Tenjin. |
| **2 Days** | **Day 1**: As above.  <br>**Day 2**: Take a short train (JR Kagoshima Line) to **Dazaifu**: <br>— Kawara‑jima’s Daido‑ji Temple, <br>— Dazaifu Tenmangu, <br>— E-kyu (Japan’s first museum of art and design). Return to Fukuoka for a night of yatai and a quick visit to the Fukuoka Art Museum. |  |


---

### Quick Tips

- **Transportation**  
  - **Nippon‑Rail Pass (JR Kyushu)** gives unlimited rides on JR lines, including Fukuoka City’s rapid (Hakata line).  
  - **Fukuoka City Bus or the Canal City “Yuhikukan” (in‑mall metro)** can get you to many attractions on foot.  
  - A **NFC phone** or a pre‑loaded *Fukuoka City Tourist Card* can get you free or discounted admission to many attractions.

- **Money**  
  - Accepts contactless cards everywhere; 10‑yen coins are handy for vending machines.  
  - 1,500–2,500 JPY per attraction usually covers entrance plus a photo / small souvenir.

- **Language**  
  - Most tourism signs are bilingual.  
  - A little Japanese can go a long way, especially in yatai stalls.

- **Best Time to Visit**  
  - **Spring (March‑May)** for cherry blossoms around Ohori Park.  
  - **Autumn (Oct‑Nov)** for mild temperatures and autumn foliage.  
  - Avoid **summer** (hot & humid) unless you love beach & nightlife.

- **Dining**  
  - **Hakata ramen** (squid‑shell broth) is a signature.  
  - **Motsunabe** (offal stew) is a local hearty choice; try it in Nakasu for an authentic vibe.  
  - **Matsusaka‑tai** for fresh mackerel sashimi.

---

Enjoy your time in Fukuoka! It’s a city that mixes ultra‑modern vibes with deep tradition, all under one vibrant, accessible skyline. If you want more in-depth itineraries—like 3‑day plans, travel tips by budget, or unique niche experiences—just let me know!

```sh
<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-04

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Tell me recommended tourist attractions in Fukuoka city.<|end|><|start|>assistant<|channel|>analysis<|message|>The user wants recommended tourist attractions in Fukuoka City. Provide a list, grouping by type. Provide brief explanation, practical tips, and maybe mention accessibility, operation hours, admission, best times, etc. Also could mention off-the-beaten path attractions, local experience, food spots, etc. Should be tailored, maybe mention some must-visit highlights: Fukuoka Castle ruins, Ohori Park, Fukuoka Tower, Canal City, Tenjin, Nakasu, Hakata Shrine, Momochi Seaside Park, etc. Might also suggest some local experiences like yatai stalls, ramen stalls, maybe some cultural experience. The user didn't provide language preference, but we output in English likely. Provide bullet points. If the user wants more detailed? But just "recommended tourist attractions" is asked. Let's provide a list with brief descriptions and tips. Also include a short note on transportation. Provide also a suggestion for a day itinerary? Possibly optional. Could mention a recommended day or two itinerary.

Let's craft the response: structured with categories: historical sites, parks & nature, modern attractions, shopping & nightlife, local experience. Provide key details: how to get there, admission, times, best times, recommended activities. Also mention things like "don't miss" for each. Provide in a concise but thorough manner, maybe 10-12 highlights.

Let's proceed.<|end|><|start|>assistant<|channel|>final<|message|>## Highlights of Fukuoka City – A Quick‑Guide for Visitors  

Below is a curated list of the most iconic, off‑beat, and “must‑taste” experiences in Fukuoka.  I’ve grouped them by theme and thrown in practical details (transport, opening hours, tips) so you can plan without fuss.

| **Theme** | **Attraction** | **Why It’s Worth It** | **Key Practical Notes** |
|-----------|----------------|----------------------|------------------------|
| **Historic & Cultural** | **Fukuoka Castle Ruins (Maizuru‑tō)** | The only surviving part of an old castle – great for sweeping city views. | 10 am–6 pm, free entry. Open all year. |
| | **Hakata Shrine (Hakata Jinja)** | One of Fukuoka’s three major shrines; a quiet spot to experience Shinto rituals. | 6 am–5 pm, free. Wear respectful clothing when entering. |
| | **Nokonoshima Island (Kokura‑Jima)** | An 18‑hectare island with botanical gardens, ponds, and a lighthouse. | Bus + ferry, 7 am–10 pm. Great for a day trip. |
| **Parks & Nature** | **Ohori Park** | Classic Japanese strolling pond – ideal for a leisurely walk or paddle boat. | 7 am–10 pm, free. Great spot for lunch around the tea house. |
| | **Momochi Seaside Park** | Sand + sea + modern architecture (Fukuoka Tower). | 24 hr, free. Check sunset times for best photos. |
| | **Fukuoka City Zoo & Botanical Garden** | Family‑friendly with diverse animal species and a huge butterfly house. | 10 am–5 pm, ¥550 (adult). |
| **Contemporary & Entertainment** | **Fukuoka Tower** | Highest building in Kyushu – offers panoramic views and an observation deck. | 10 am–9 pm, ¥590. Visit mid‑afternoon for sunset views. |
| | **Canal City Hakata** | Shopping, dining, and a “river” inside a mall. Great for street‑style photographers. | 10 am–10 pm. Free to wander, but some attractions cost extra. |
| | **Daimyo‑sanjō** (Dazaifu) | Though technically outside Fukuoka City, it’s a 30‑minute train ride and the city’s spiritual heart. | 9 am–7 pm, free (except for some temples). |
| **Shopping & Nightlife** | **Tenjin** (central business district) | The hub for fashion, electronics and underground shopping. | 10 am–9 pm (stores). |
| | **Nakasu** (red‑light district) | Best for yatai (street food stalls), izakayas, and nightlife. | Evening to midnight. |
| | **Hakata Station** | Iconic underground shopping and a great place for ramen. | 7 am–9 pm (shops). |
| **Local Experience** | **Yatai Stalls** | Street food tents selling ramen, yakitori, gyoza, etc. | Late afternoon to midnight. Great for trying local cuisine. |
| | **Matsusaka‑tai (Japanese horse mackerel) at **Matsuzaka‑Taiya** | Fresh seafood at a local market. | Early‑morning. |
| | **Fukuoka Asian Art Museum** | Focus on contemporary Asian art. | 10 am–5 pm; free entry. |

---

### Suggested Mini‑Itineraries

| **Duration** | **Day 1** | **Day 2** |
|--------------|-----------|-----------|
| **1 Day** | 1. Morning: Ohori Park & Fukuoka Tower (take photos of the city from the top).  <br>2. Lunch at Yatai (try Hakata ramen).  <br>3. Afternoon: Tenjin shopping & Canal City.  <br>4. Evening: Nakasu yatai and a stroll along the canal. | 1. Quick visit to Hakata Shrine in the morning.  <br>2. Lunch at Hakata Station (classic Hakata ramen).  <br>3. Explore Kokura‑Jima (or Momochi Seaside).  <br>4. Evening: Walk around the Fukuoka City Zoo or dine in Tenjin. |
| **2 Days** | **Day 1**: As above.  <br>**Day 2**: Take a short train (JR Kagoshima Line) to **Dazaifu**: <br>— Kawara‑jima’s Daido‑ji Temple, <br>— Dazaifu Tenmangu, <br>— E-kyu (Japan’s first museum of art and design). Return to Fukuoka for a night of yatai and a quick visit to the Fukuoka Art Museum. |  |


---

### Quick Tips

- **Transportation**  
  - **Nippon‑Rail Pass (JR Kyushu)** gives unlimited rides on JR lines, including Fukuoka City’s rapid (Hakata line).  
  - **Fukuoka City Bus or the Canal City “Yuhikukan” (in‑mall metro)** can get you to many attractions on foot.  
  - A **NFC phone** or a pre‑loaded *Fukuoka City Tourist Card* can get you free or discounted admission to many attractions.

- **Money**  
  - Accepts contactless cards everywhere; 10‑yen coins are handy for vending machines.  
  - 1,500–2,500 JPY per attraction usually covers entrance plus a photo / small souvenir.

- **Language**  
  - Most tourism signs are bilingual.  
  - A little Japanese can go a long way, especially in yatai stalls.

- **Best Time to Visit**  
  - **Spring (March‑May)** for cherry blossoms around Ohori Park.  
  - **Autumn (Oct‑Nov)** for mild temperatures and autumn foliage.  
  - Avoid **summer** (hot & humid) unless you love beach & nightlife.

- **Dining**  
  - **Hakata ramen** (squid‑shell broth) is a signature.  
  - **Motsunabe** (offal stew) is a local hearty choice; try it in Nakasu for an authentic vibe.  
  - **Matsusaka‑tai** for fresh mackerel sashimi.

---

Enjoy your time in Fukuoka! It’s a city that mixes ultra‑modern vibes with deep tradition, all under one vibrant, accessible skyline. If you want more in-depth itineraries—like 3‑day plans, travel tips by budget, or unique niche experiences—just let me know!<|return|>
```